In [ ]:
import numpy as np
import os
import json
import cv2
from mlboardclient.api import client
import os

In [ ]:
src_dir = os.environ['CODE_DIR']+'/coco'
src_dir

In [ ]:
with open(src_dir + '/annotations/instances_train2017.json') as f:
    data = json.load(f)

In [ ]:
data = data['annotations']

In [ ]:
len(data)

In [ ]:
train_dir = os.environ['TRAINING_DIR']+'/data_tmp'
train_dir

In [ ]:
import shutil

In [ ]:
shutil.rmtree(train_dir,True)

In [ ]:
os.mkdir(train_dir)

In [ ]:
images_dir = train_dir+'/images'
mask_dir = train_dir+'/masks'

In [ ]:
os.mkdir(images_dir)
os.mkdir(mask_dir)

In [ ]:
limit_pictures = int(os.environ.get('LIMIT_PIC',10))
f'Number pictures: {limit_pictures}'

In [ ]:
i = 0
count = len(data)
if limit_pictures<0:
    limit_pictures = len(data)
for a in data:
    if i>=limit_pictures:
        break
    if a['category_id'] == 1 and a['iscrowd'] == 0:
        name = '{:012d}.jpg'.format(a['image_id'])
        fname = src_dir + '/train2017/{}'.format(name)
        segmentation = a['segmentation']
        if len(segmentation) < 4 and len(segmentation)>0:
            area = a['area']
            if os.path.exists(fname):
                img = cv2.imread(fname, cv2.IMREAD_COLOR)
                img_area = img.shape[0]*img.shape[1]
                if area>img_area*0.1:
                    m = np.zeros((img.shape[0], img.shape[1]), np.uint8)
                    cv2.imwrite(images_dir+'/'+name,img)
                    for s in segmentation:
                        p = np.array(s, np.int32)
                        p = np.reshape(p, (1, int(p.shape[0] / 2), 2))
                        m = cv2.fillPoly(m, p, color=(255, 255, 255))
                    mask_file = mask_dir+'/'+name
                    if os.path.exists(mask_file):
                        pm = cv2.imread(mask_file)
                        if len(pm.shape)>2:
                            pm = pm[:,:,0]
                        m = np.maximum(m,pm)
                    cv2.imwrite(mask_file,m)
                    if i%1000==0:
                        print(f'Proccesing: {i} of {count}')
                    i+=1

In [ ]:
import glob

In [ ]:
files = list(glob.glob(images_dir+'/*.jpg'))


In [ ]:
import random


In [ ]:
files = random.sample(files, k=4)

In [ ]:
import PIL.Image
import matplotlib.pyplot as plt
for f in files:
    print(f)
    name = os.path.basename(f)
    mask = mask_dir+'/'+name
    img = cv2.imread(f)
    mask = cv2.imread(mask)
    img[mask<255]=0
    img = PIL.Image.fromarray(img[:,:,::-1])
    plt.imshow(img)
    plt.show()

In [ ]:
client.update_task_info({'num-files': i})

In [ ]:
m = client.Client()

In [ ]:
workspace = os.environ.get('WORKSPACE_NAME')

In [ ]:
name = 'people-mask'
build = int(os.environ.get('BUILD_ID',0))
version = f'1.0.{build}'
m.datasets.push(workspace,name,version,train_dir,type='dataset',create=True)
reference = '#/{}/catalog/dataset/{}/versions/{}'.format(workspace, name, version)
if build>0:
    client.update_task_info({'data_reference':reference})